In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Adjust the input size based on your image size
        self.fc2 = nn.Linear(128, 5)  # Output layer for 5 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Dataset loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train_dataset = datasets.ImageFolder(root='e:/Diabetic_Retinopathy_Detection/input/train', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Model, loss function, and optimizer
model = CustomCNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete!")

# Save the trained model
torch.save(model.state_dict(), 'custom_cnn.pth')


In [5]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm  # Import tqdm

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 100

# Dataset loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

train_dataset = datasets.ImageFolder(root='e:/Diabetic_Retinopathy_Detection/input/train', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Model, loss function, and optimizer
model = CustomCNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Wrap train_loader with tqdm for a progress bar
    loop = tqdm(train_loader, leave=True)
    for images, labels in loop:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update the progress bar
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=loss.item())
    
    # Optional: You can add another print statement here if you want more info

print("Training complete!")

# Save the trained model
torch.save(model.state_dict(), 'custom_cnn.pth')


Epoch [10/10]: 100%|██████████| 113/113 [03:45<00:00,  2.00s/it, loss=0.781]


Training complete!


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hyperparameters
batch_size = 64

# Dataset loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit the model input
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder(root='e:/Diabetic_Retinopathy_Detection/input/test', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Load the trained model
model = CustomCNN()
model.load_state_dict(torch.load('custom_cnn.pth'))
model.eval()  # Set model to evaluation mode

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():  # No need to calculate gradients
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total}%')


In [6]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm  # Import tqdm

# Hyperparameters
batch_size = 64

# Dataset loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit the model input
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder(root='e:/Diabetic_Retinopathy_Detection/input/test', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Load the trained model
model = CustomCNN()
model.load_state_dict(torch.load('custom_cnn.pth'))
model.eval()  # Set model to evaluation mode

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():  # No need to calculate gradients
    # Wrap test_loader with tqdm for a progress bar
    loop = tqdm(test_loader, leave=True)
    for images, labels in loop:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Optional: Update the progress bar with additional info
        loop.set_postfix(accuracy=f'{100 * correct / total:.2f}%')

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')


100%|██████████| 17/17 [01:44<00:00,  6.13s/it, accuracy=36.44%]

Accuracy of the model on the test images: 36.44%
